In [3]:
# import packages here
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
#print('OpenCv Version:',cv2.__version__)
#NOTE: OpenCv Version: 3.4.2
import os
#os.sys.path

Barn: This section load data
1. train data, trian label
2. test data, test label

In [ ]:
class_names = [name[11:] for name in glob.glob('data/train/*')]
class_names = dict(zip(range(0,len(class_names)), class_names))
print (class_names)
def load_dataset(path, num_per_class=-1):
    data = []
    labels = []
    for id, class_name in class_names.items():
        img_path_class = glob.glob(path + class_name + '/*.jpg')
        if num_per_class > 0:
            img_path_class = img_path_class[:num_per_class]
        labels.extend([id]*len(img_path_class))
        for filename in img_path_class:
            data.append(cv2.imread(filename, 0))
    return data, labels

# load training dataset
train_data, train_label = load_dataset('data/train/', 100)
train_num = len(train_label)
print (train_num)
# load testing dataset
test_data, test_label = load_dataset('data/test/', 100)
test_num = len(test_label)

Dummy Model Template
1. Extract Feature Function
2. Train Function
3. Predict Function

In [12]:
# feature extraction
def extract_feat(raw_data):
    feat_dim = 1000
    feat = np.zeros((len(raw_data), feat_dim), dtype=np.float32)
    for i in range(0,feat.shape[0]):
        feat[i] = np.reshape(raw_data[i], (raw_data[i].size))[:feat_dim] # dummy implemtation
        
    return feat

train_feat = extract_feat(train_data)
test_feat = extract_feat(test_data)

# model training: take feature and label, return model
def train(X, Y):
    return 0 # dummy implementation

# prediction: take feature and model, return label
def predict(model, x):
    return np.random.randint(15) # dummy implementation

# evaluation
predictions = [-1]*len(test_feat)
for i in range(0, test_num):
    predictions[i] = predict(None, test_feat[i])
    
accuracy = sum(np.array(predictions) == test_label) / float(test_num)

print ("The accuracy of my dummy model is {:.2f}%".format(accuracy*100))

The accuracy of my dummy model is 6.20%


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# train model
def trainKNN(data, labels, k):
    neigh = KNeighborsClassifier(n_neighbors=k, p=2)
    neigh.fit(data, labels) 
    return neigh

## Bag of SIFT Representation + Nearest Neighbor Classifer
1. __*Vectorise Image*__ by converting image to dense SIFT
    - computeSIFT(data)
        - return SHIFT data x
2. __*build BoW*__ presentation from SIFT of training images
    - clusterFeatures(all_train_desc, k)
        - return k means
        - K means clustering to for k number of means (k features) from SIFT
3. __*form training set histograms*__ for each training image using BoW representation 
    - def formTrainingSetHistogram(x_train, kmeans, k):
        - return histogram of features
        - using SIFT predict closest k mean and fill up feature histogram
3. __*build histograms for test set*__  and __*predict using KNN*__  
    - def predictKMeans(kmeans, scaler, x_test, train_hist, train_label, k):
        - return test set prediction
        - Use KNN(K nearest neighbours) different from K mean clustering.
            - k-NN is a supervised algorithm used for classification
            - For each evaluation matrix, Decide upon the value of k. Here k refers to the number of               closest neighbors we will consider while doing the majority voting of target labels.
            - Run k-NN a few times, changing k and checking the evaluation measure.
            - In each iteration, k neighbors vote, majority vote wins and becomes the ultimate    prediction
            - Optimize k by picking the one with the best evaluation measure.
            - Once you’ve chosen k, use the same training set and now create a new test set with the people’s ages and incomes that you have no labels for, and want to predict.
        - Train KNN using train_hist and train_label
        - Use trained KNN to predict test_hist's label     
4. __*evaluate accuracy*__  of predict test_hist's label
    - def accuracy(predict_label, test_label):
        - return correct label/ total test label
        

In [13]:
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [14]:
# compute dense SIFT 
def computeSIFT(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 15
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
        dense_feat = sift.compute(img, kp)
        x.append(dense_feat[1])
        
    return x

# extract dense sift features from training images
x_train = computeSIFT(train_data)
x_test = computeSIFT(test_data)

all_train_desc = []
for i in range(len(x_train)):
    for j in range(x_train[i].shape[0]):
        all_train_desc.append(x_train[i][j,:])

all_train_desc = np.array(all_train_desc)

AttributeError: module 'cv2.cv2' has no attribute 'xfeatures2d'

In [7]:
# build BoW presentation from SIFT of training images 
def clusterFeatures(all_train_desc, k):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(all_train_desc)
    return kmeans

In [8]:
# form training set histograms for each training image using BoW representation
def formTrainingSetHistogram(x_train, kmeans, k):
    train_hist = []
    for i in range(len(x_train)):
        data = copy.deepcopy(x_train[i])
        predict = kmeans.predict(data)
        train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        
    return np.array(train_hist)


# build histograms for test set and predict
def predictKMeans(kmeans, scaler, x_test, train_hist, train_label, k):
    # form histograms for test set as test data
    test_hist = formTrainingSetHistogram(x_test, kmeans, k)
    
    # make testing histograms zero mean and unit variance
    test_hist = scaler.transform(test_hist)
    
    # Train model using KNN
    knn = trainKNN(train_hist, train_label, k)
    predict = knn.predict(test_hist)
    return np.array([predict], dtype=np.array([test_label]).dtype)
    

def accuracy(predict_label, test_label):
    return np.mean(np.array(predict_label.tolist()[0]) == np.array(test_label))

### Accuracy with different number of clusters

In [9]:
k = [10, 15, 20, 25, 30, 35, 40]
for i in range(len(k)):
    kmeans = clusterFeatures(all_train_desc, k[i])
    train_hist = formTrainingSetHistogram(x_train, kmeans, k[i])
    
    # preprocess training histograms
    scaler = preprocessing.StandardScaler().fit(train_hist)
    train_hist = scaler.transform(train_hist)
    
    predict = predictKMeans(kmeans, scaler, x_test, train_hist, train_label, k[i])
    res = accuracy(predict, test_label)
    print("k =", k[i], ", Accuracy:", res*100, "%")

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Bag of SIFT Representation + one-vs-all SVMs

1. (Same)__*Vectorise Image*__ by converting image to dense SIFT
    - computeSIFT(data)
        - return SHIFT data x
2. (Same)__*build BoW*__ presentation from SIFT of training images
    - clusterFeatures(all_train_desc, k)
        - return k means
        - K means clustering to for k number of means (k features) from SIFT
3. (Same)__*form training and test set histograms*__ for each training image using BoW representation 
    - def formTrainingSetHistogram(x_train, kmeans, k):
        - return histogram of features
        - using SIFT predict closest k mean and fill up feature histogram
3. __*Use 1 v All SVM (Support Vector Machine) to train and predict*__
     - clf = LinearSVC(random_state=0, C=c)
     - clf.fit(train_hist, train_label)
     - predict = clf.predict(test_hist)
         - Large Value of parameter C => small margin
         - Small Value of paramerter C => Large margin
             - find optimal smallest c (largest margin) value for training data set so that misclassification of test data set is lower. In order words, it gerneralised the SVM model for other data sets other than training data set source: Medium
4. (Same)__*evaluate accuracy*__  of predict test_hist's label
    - def accuracy(predict_label, test_label):
        - return correct label/ total test label

In [10]:
from sklearn.svm import LinearSVC

In [11]:
k = 60
kmeans = clusterFeatures(all_train_desc, k)

# form training and testing histograms
train_hist = formTrainingSetHistogram(x_train, kmeans, k)
test_hist = formTrainingSetHistogram(x_test, kmeans, k)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [12]:
# normalize histograms
scaler = preprocessing.StandardScaler().fit(train_hist)
train_hist = scaler.transform(train_hist)
test_hist = scaler.transform(test_hist)

NameError: name 'train_hist' is not defined

### Train one-vs-all SVMs using sklearn

In [13]:
for c in np.arange(0.0001, 0.1, 0.00198):
    clf = LinearSVC(random_state=0, C=c)
    clf.fit(train_hist, train_label)
    predict = clf.predict(test_hist)
    print ("C =", c, ",\t Accuracy:", np.mean(predict == test_label)*100, "%")

NameError: name 'train_hist' is not defined

### We can train 15 SVM classifiers manually and get same result

In [14]:
y_train_global = np.zeros((len(train_label), 1))
y = copy.deepcopy(y_train_global)

NameError: name 'copy' is not defined

In [15]:

y_predict = np.zeros((len(test_label), 1))
for i in range(len(test_label)):
    index = 0
    test = np.array([test_hist[i,:]]).T
    for j in range(len(class_names)):
        y = copy.deepcopy(y_train_global)
        y[index:index+100, 0:1] = np.ones((100,1))
        clf = LinearSVC(random_state=0, C=0.06148)
        clf.fit(train_hist, y.ravel())
        if j == 0:
            maxScore = np.dot(clf.coef_, test) + clf.intercept_
            y_predict[i, 0:1] = j
        elif np.dot(clf.coef_, test) + clf.intercept_ > maxScore:
            maxScore = np.dot(clf.coef_, test) + clf.intercept_
            y_predict[i, 0:1] = j
        index = index + 100

In [16]:
print ("Accuracy:", np.mean(y_predict.ravel() == test_label)*100, "%")

Accuracy: nan %


C:\Users\Barnabas Lim\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Barnabas Lim\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(np.array([test_label]).T, y_predict)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(18, 6))
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure(figsize=(18, 6))
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

IndexError: index 0 is out of bounds for axis 0 with size 0

## Improve performance with Spatial Pyramid Matching
1. (Same done)__*Vectorise Image*__ by converting image to dense SIFT
    - computeSIFT(data)
        - return SHIFT data x
2. (Same 200 features SHIFT data)__*build BoW*__ presentation from SIFT of training images
    - clusterFeatures(all_train_desc, k)
        - return k means
        - K means clustering to for k number of means (k features) from SIFT
3. (different)__*form training and test set feature histograms*__ for each training image using BoW representation 
    - __*getHistogramSPM(L, data, kmeans, k)*__
        - return SPM Histogram
            - return combined SPM Histogram
        - - ↑ __*getImageFeaturesSPM(L, img, kmeans, k)*__
            - return SPM Histogram for each all level of spaces in the spacial pyramid.
            - For L=0: 1 image, L=1: 4image, L=2: 16image
        - - - ↑ __*def extract_denseSIFT(img)*__
              - return SHIFT data for each elements of images in each elvel
              - DSIFT_STEP_SIZE=2 different from that of __*computeSIFT(data)*__ that has step size of 15.
          
3. (same) __*Use 1 v All SVM (Support Vector Machine) to train and predict*__
     - clf = LinearSVC(random_state=0, C=c)
     - clf.fit(train_hist, train_label)
     - predict = clf.predict(test_hist)
         - Large Value of parameter C => small margin
         - Small Value of paramerter C => Large margin
             - find optimal smallest c (largest margin) value for training data set so that misclassification of test data set is lower. In order words, it gerneralised the SVM model for other data sets other than training data set source: Medium
4. (Same)__*evaluate accuracy*__  of predict test_hist's label
    - def accuracy(predict_label, test_label):
        - return correct label/ total test label

In [5]:
import math

def extract_denseSIFT(img):
    DSIFT_STEP_SIZE = 2
    sift = cv2.xfeatures2d.SIFT_create()
    disft_step_size = DSIFT_STEP_SIZE
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img.shape[0], disft_step_size)
                for x in range(0, img.shape[1], disft_step_size)]

    descriptors = sift.compute(img, keypoints)[1]
    
    #keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors



# form histogram with Spatial Pyramid Matching upto level L with codebook kmeans and k codewords
def getImageFeaturesSPM(L, img, kmeans, k):
    W = img.shape[1]
    H = img.shape[0]   
    h = []
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):                
                desc = extract_denseSIFT(img[y:y+h_step, x:x+w_step])                
                #print("type:",desc is None, "x:",x,"y:",y, "desc_size:",desc is None)
                predict = kmeans.predict(desc)
                histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    # normalize hist
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist


# get histogram representation for training/testing data
def getHistogramSPM(L, data, kmeans, k):    
    x = []
    for i in range(len(data)):        
        hist = getImageFeaturesSPM(L, data[i], kmeans, k)        
        x.append(hist)
    return np.array(x)

In [19]:
k = 200
kmeans = clusterFeatures(all_train_desc, k)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [20]:
train_histo = getHistogramSPM(2, train_data, kmeans, k)
test_histo = getHistogramSPM(2, test_data, kmeans, k)

NameError: name 'kmeans' is not defined

In [21]:
# train SVM
for c in np.arange(0.000307, 0.001, 0.0000462):
    clf = LinearSVC(random_state=0, C=c)
    clf.fit(train_histo, train_label)
    predict = clf.predict(test_histo)
    print ("C =", c, ",\t\t Accuracy:", np.mean(predict == test_label)*100, "%")

NameError: name 'train_histo' is not defined